# Pré-processamento dos dados das exportações do RN
## Etapa 3 - Engenharia de atributos

<img height="200" src="https://share-eric.eu/fileadmin/_processed_/b/5/csm_dataheader_ec9ee966be.jpg" width="600"/>

# Carregando base de dados das exportações do RN após etapa 2 de pre-processamento

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as stats


In [39]:
import pandas as pd

# Carrega excel em um dataframe
dataset_exportacoes_sem_outliers = pd.read_csv('../../../data/interim/preprocessing/02_dataset_expo_rn_NO_OUTLIERS.csv')
dataset_exportacoes_outubro = pd.read_csv('../../../data/interim/preprocessing/01_dataset_expo_rn_outubro_data_cleaning.csv')
dataset_exportacoes_outubro_sem_outliers = pd.read_csv('../../../data/interim/preprocessing/02_dataset_expo_rn_outubro_NO_OUTLIERS.csv')
dataset_exportacoes_com_outliers = pd.read_csv('../../../data/interim/preprocessing/02_dataset_expo_rn_OUTLIERS.csv')
dataset_exportacoes_com_outliers

,Bloco Econômico,Países,Via,URF,Código NCM,Descrição NCM,Código SH6,Descrição SH6,Código SH4,Descrição SH4,Código SH2,Descrição SH2,Código Seção,Descrição Seção,Unidade estatística,Valor US$ FOB,Quilograma Líquido,Quantidade Estatística,Data
0,3,117,0,68,13021999,Outros sucos e extratos vegetais,130219,Sucos e extratos de outros vegetais (mamão sec...,57,495,12,27,1,17,8,3,0,0,2024-09-01
1,3,122,0,68,49111090,"Outros impressos publicitários, catálogos come...",491110,"Impressos publicitários, catálogos comerciais ...",229,383,44,33,9,12,8,5,0,0,2024-01-01
2,3,122,0,68,96110000,"Carimbos, incluindo os datadores e numeradores...",961100,"Carimbos, datadores, numeradores, sinetes e ar...",549,98,86,46,18,7,8,5,0,0,2024-01-01
3,3,122,0,68,61071200,"Cuecas e ceroulas, de malha, de uso masculino,...",610712,"Cuecas e ceroulas, de malha, de fibras sintéti...",276,153,55,83,10,6,5,5,0,1,2024-01-01
4,3,122,0,68,61072900,"Camisolões e pijamas, de malha, de uso masculi...",610729,"Camisolões e pijamas, de malha, de outras maté...",276,153,55,83,10,6,5,5,0,1,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57130,6,108,4,41,25010090,"Outros tipos de sal, cloreto de sódio puro e á...",250100,Sal (incluídos o sal de mesa e o sal desnatura...,92,484,23,76,5,18,8,424298,24958710,24959,1997-05-01
57131,1,51,4,41,25010011,"Sal marinho, a granel, sem agregados",250100,Sal (incluídos o sal de mesa e o sal desnatura...,92,484,23,76,5,18,8,341000,31000000,31000,1997-10-01
57132,1,51,4,41,25010011,"Sal marinho, a granel, sem agregados",250100,Sal (incluídos o sal de mesa e o sal desnatura...,92,484,23,76,5,18,8,396000,36000000,36000,1997-08-01
57133,1,51,4,41,25010011,"Sal marinho, a granel, sem agregados",250100,Sal (incluídos o sal de mesa e o sal desnatura...,92,484,23,76,5,18,8,436680,36000000,36000,1997-02-01


In [40]:
dataset_exportacoes_com_outliers.dtypes

Bloco Econômico            int64
Países                     int64
Via                        int64
URF                        int64
Código NCM                 int64
Descrição NCM             object
Código SH6                 int64
Descrição SH6             object
Código SH4                 int64
Descrição SH4              int64
Código SH2                 int64
Descrição SH2              int64
Código Seção               int64
Descrição Seção            int64
Unidade estatística        int64
Valor US$ FOB              int64
Quilograma Líquido         int64
Quantidade Estatística     int64
Data                      object
dtype: object

# Condensando dataset e deixando soma mensal das exportações de cada produto

In [41]:
import pandas as pd
df = dataset_exportacoes_com_outliers.copy()

# Converter a coluna "Data" para datetime, caso ainda não esteja nesse formato
df['Data'] = pd.to_datetime(df['Data'])

# Criar uma coluna com o ano e o mês (no formato AAAA-MM) para o agrupamento
df['Ano_Mes'] = df['Data'].dt.to_period('M')

# Agrupar os dados por Ano_Mes e Código SH6, somando o valor FOB
df_mes = df.groupby(['Ano_Mes', 'Código SH6'])['Valor US$ FOB'].sum().reset_index()

# Renomear colunas para deixar o resultado mais claro
df_mes.rename(columns={'Ano_Mes': 'Data', 'Código SH6': 'Código SH6', 'Valor US$ FOB': 'Valor US$ FOB Mensal'}, inplace=True)

# Visualizar o resultado
df_mes


,Data,Código SH6,Valor US$ FOB Mensal
0,1997-01,30269,3395
1,1997-01,30611,600068
2,1997-01,51199,5220
3,1997-01,80132,1773030
4,1997-01,80300,62229
...,...,...,...
19838,2024-09,820510,2775
19839,2024-09,841221,1401
19840,2024-09,842519,8125
19841,2024-09,854929,134857


In [42]:
import pandas as pd

# Criar uma cópia do dataset
df = dataset_exportacoes_com_outliers.copy()

# Converter a coluna "Data" para datetime, caso ainda não esteja nesse formato
df['Data'] = pd.to_datetime(df['Data'])

# Criar uma coluna com o ano para o agrupamento
df['Ano'] = df['Data'].dt.year

# Agrupar os dados por Ano e Código SH6, somando o valor FOB
df_anual = df.groupby(['Ano', 'Código SH6'])['Valor US$ FOB'].sum().reset_index()

# Renomear colunas para deixar o resultado mais claro
df_anual.rename(columns={'Ano': 'Ano', 'Código SH6': 'Código SH6', 'Valor US$ FOB': 'Valor US$ FOB Anual'}, inplace=True)

# Visualizar o resultado
df_anual


,Ano,Código SH6,Valor US$ FOB Anual
0,1997,30269,1345963
1,1997,30379,299775
2,1997,30611,3664532
3,1997,30623,401
4,1997,50690,33129
...,...,...,...
4959,2024,940511,301
4960,2024,950300,6
4961,2024,960190,1931
4962,2024,960621,2518


# Exportando bases de dados apos engenharia de atributos

In [43]:
dataset_exportacoes_outubro.dtypes

Países                     int64
Bloco Econômico            int64
Via                        int64
URF                        int64
Código NCM                 int64
Descrição NCM             object
Código SH6                 int64
Descrição SH6             object
Código SH4                 int64
Descrição SH4              int64
Código SH2                 int64
Descrição SH2              int64
Código Seção               int64
Descrição Seção            int64
Unidade estatística        int64
Valor US$ FOB              int64
Quilograma Líquido         int64
Quantidade Estatística     int64
Data                      object
dtype: object

In [49]:
df_mes.to_csv("../../../data/interim/preprocessing/03_dataset_expo_rn_fob_mensal_OUTLIERS.csv", index=False)
df_anual.to_csv("../../../data/interim/preprocessing/03_dataset_expo_rn_fob_anual_OUTLIERS.csv", index=False)
#dataset_exportacoes_outubro.to_csv("../../../data/interim/preprocessing/03_dataset_expo_rn_outubro_featured.csv", index=False)

In [47]:
df_mes

,Data,Código SH6,Valor US$ FOB Mensal
0,1997-01,30269,3395
1,1997-01,30611,600068
2,1997-01,51199,5220
3,1997-01,80132,1773030
4,1997-01,80300,62229
...,...,...,...
19838,2024-09,820510,2775
19839,2024-09,841221,1401
19840,2024-09,842519,8125
19841,2024-09,854929,134857
